In [1]:
import tensorflow as tf
import tensorflow.contrib.tensorrt as trt
import numpy as np
import cv2
from IPython.display import Image as DisplayImage


from IPython.display import display
from IPython.display import clear_output

from imutils.video import VideoStream
import imutils


%matplotlib inline
import matplotlib.pyplot as plt

from time import sleep
import multiprocessing as mp
import matplotlib.patches as patches


import time
import IPython
import io



In [2]:
from PIL import Image
from PIL import ImageDraw

In [3]:
pb_fname = "/home/pi/python/ssd_inception_v2_coco_trt.pb"

In [4]:
def showarray(a, fmt='jpeg'):
    f = io.BytesIO()
    #Image.fromarray(a).save(f, fmt)
    a.save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

In [5]:
def get_frozen_graph(graph_file):
    """Read Frozen Graph file from disk."""
    with tf.gfile.FastGFile(graph_file, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    return graph_def


trt_graph = get_frozen_graph(pb_fname)

Instructions for updating:
Use tf.gfile.GFile.


In [6]:
from PIL import ImageFont

In [ ]:
input_names = ['image_tensor']
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True

tf_sess = tf.Session(config=tf_config)

tf.import_graph_def(trt_graph, name='')

tf_input = tf_sess.graph.get_tensor_by_name(input_names[0] + ':0')
tf_scores = tf_sess.graph.get_tensor_by_name('detection_scores:0')
tf_boxes = tf_sess.graph.get_tensor_by_name('detection_boxes:0')
tf_classes = tf_sess.graph.get_tensor_by_name('detection_classes:0')
tf_num_detections = tf_sess.graph.get_tensor_by_name('num_detections:0')

In [ ]:
def camThread():

    camera_width = 320
    camera_height = 240
    vidfps = 30

    #cam = cv2.VideoCapture(0)
    cam = cv2.VideoCapture("nvarguscamerasrc ! video/x-raw(memory:NVMM), width=(int)640, height=(int)480,format=(string)NV12, framerate=(fraction)30/1 ! nvvidconv ! video/x-raw, format=(string)BGRx ! videoconvert ! video/x-raw, format=(string)BGR ! appsink")

    #cam.set(cv2.CAP_PROP_FPS, vidfps)
    #cam.set(cv2.CAP_PROP_FRAME_WIDTH, camera_width)
    #cam.set(cv2.CAP_PROP_FRAME_HEIGHT, camera_height)
    font = ImageFont.truetype("arial.ttf", 20)
    while True:
        try:
            t1 = time.perf_counter()

            ret, color_image = cam.read()
            if not ret:
                #print("no image")
                continue
            frames = color_image

            prepimg = color_image[:, :, ::-1].copy()
            prepimg = Image.fromarray(prepimg)
            draw = ImageDraw.Draw(prepimg)
            tinf = time.perf_counter()
            image_resized = np.array(prepimg.resize((300, 300)))
            #image_resized = np.array(image.resize((300, 300)))
            #image_resized = np.array(image.resize((300, 300)))
            image = np.array(prepimg)
            #print("Hello detect !!!!")
            t1 = time.time()
            #cores, boxes, classes, num_detections = tf_sess.run([tf_scores, tf_boxes, tf_classes, tf_num_detections], feed_dict={tf_input: image_resized[None, ...]})
            scores, boxes, classes, num_detections = tf_sess.run([tf_scores, tf_boxes, tf_classes, tf_num_detections], feed_dict={tf_input: image_resized[None, ...]})
            boxes = boxes[0] # index by 0 to remove batch dimension
            scores = scores[0]
            classes = classes[0]
            num_detections = num_detections[0]
            #print(num_detections)
            for i in range(num_detections.astype(int)):
                # scale box to image coordinates
                #print(boxes[i])
                box = boxes[i] * np.array([image.shape[0], image.shape[1], image.shape[0], image.shape[1]])

                # display rectangle
                #patch = patches.Rectangle((box[1], box[0]), box[3] - box[1], box[2] - box[0], color='g', alpha=0.3)
                #ax.add_patch(patch)

                draw.rectangle([box[1], box[0], box[3], box[2]], outline='red')
                s='%d (%0.2f) ' % (classes[i], scores[i])

                draw.text((box[1], box[2]-20), s , fill='green',font=font)

            t2 = time.time()
            fps = 1/(t2-t1)
            fps_str = 'FPS = %.2f' % fps
            draw.text((10,220), fps_str , fill='green', font=font)
            showarray(prepimg)
            clear_output(wait=True)
            res = None
        except KeyboardInterrupt:
            cam.release() 
            break


In [ ]:
camThread()

In [ ]:
def inferencer(results, frameBuffer):

    #engine = DetectionEngine(model)
    print("Hello!!!!")
        
    while True:

        if frameBuffer.empty():
            #print("Buffer gone")
            continue
       
        # Run inference.
        color_image = frameBuffer.get()
        prepimg = color_image[:, :, ::-1].copy()
        prepimg = Image.fromarray(prepimg)
        draw = ImageDraw.Draw(prepimg)
        tinf = time.perf_counter()
        image_resized = np.array(prepimg.resize((300, 300)))
        #image_resized = np.array(image.resize((300, 300)))
        #image_resized = np.array(image.resize((300, 300)))
        image = np.array(prepimg)
        #print("Hello detect !!!!")
        t1 = time.time()
        #cores, boxes, classes, num_detections = tf_sess.run([tf_scores, tf_boxes, tf_classes, tf_num_detections], feed_dict={tf_input: image_resized[None, ...]})
        scores, boxes, classes, num_detections = tf_sess.run([tf_scores, tf_boxes, tf_classes, tf_num_detections], feed_dict={tf_input: image_resized[None, ...]})
        print(num_detections)

            
            
        t2 = time.time()
        fps = 1/(t2-t1)
        fps_str = 'FPS = %.2f' % fps
        draw.text((10,220), fps_str , fill='green')
        showarray(prepimg)
        clear_output(wait=True)
        #print("Hello detect !!!!")
        #print(time.perf_counter() - tinf, "sec")
        #results.put(ans)

In [ ]:
try:
    frameBuffer = mp.Queue(10)
    results = mp.Queue()

    # Start streaming
    p = mp.Process(target=camThread, args=(results, frameBuffer), daemon=True)
    p.start()
    processes.append(p)


    # Activation of inferencer
    p = mp.Process(target=inferencer, args=(results, frameBuffer), daemon=True)
    p.start()
    processes.append(p)

    while True:
        sleep(1)

finally:
    for p in range(len(processes)):
        processes[p].terminate()


In [ ]:
tf_sess.close()